In [1]:
import pandas as pd 
import numpy as np
import ivolatility as ivol 
import yfinance as yf
from datetime import datetime, timedelta
import time
import os
ivol.setLoginParams(apiKey='neHi06s1WwGL40Uc')

In [2]:
# EOD Equity Options Raw IV (search by parameters)
getMarketData = ivol.setMethod('/equities/eod/stock-opts-by-param')

marketData = getMarketData(symbol='SPX',tradeDate='2024-08-05',dteFrom=0,dteTo=0,deltaFrom=-1000,deltaTo=1000,cp='C')

In [3]:
marketData

,c_date,option_symbol,dte,stocks_id,expiration_date,call_put,price_strike,price_open,price_high,price_low,...,gamma,theta,vega,rho,Ask,Bid,underlying_price,calc_OTM,option_id,is_settlement
0,2024-08-05,SPXW 240805C02200000,0,9327,2024-08-05,C,2200.0,2990.00,2990.00,2990.00,...,0.0,0.0,0.0,0.0,3004.20,2980.3,5192.005,-57.63,133236043,0
1,2024-08-05,SPXW 240805C02400000,0,9327,2024-08-05,C,2400.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,2804.20,2780.3,5192.005,-53.78,133236045,0
2,2024-08-05,SPXW 240805C02600000,0,9327,2024-08-05,C,2600.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,2604.20,2580.3,5192.005,-49.92,133236047,0
3,2024-08-05,SPXW 240805C02800000,0,9327,2024-08-05,C,2800.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,2404.20,2380.3,5192.005,-46.07,133236049,0
4,2024-08-05,SPXW 240805C03000000,0,9327,2024-08-05,C,3000.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,2198.80,2180.3,5192.005,-42.22,133236051,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,2024-08-05,SPXW 240805C06400000,0,9327,2024-08-05,C,6400.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.05,0.0,5192.005,23.27,133236161,0
209,2024-08-05,SPXW 240805C06600000,0,9327,2024-08-05,C,6600.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.05,0.0,5192.005,27.12,133236163,0
210,2024-08-05,SPXW 240805C06800000,0,9327,2024-08-05,C,6800.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.05,0.0,5192.005,30.97,133236165,0
211,2024-08-05,SPXW 240805C07000000,0,9327,2024-08-05,C,7000.0,0.05,0.05,0.05,...,0.0,0.0,0.0,0.0,0.05,0.0,5192.005,34.82,133236167,0


In [10]:
marketData.to_excel('SPX.xlsx')

In [5]:
def calculate_friday_differences(given_date):
    # Ensure given_date is a datetime object
    if isinstance(given_date, str):
        given_date = datetime.strptime(given_date, "%Y-%m-%d")

    # Calculate the last Friday in the future that is 23 to 30 days away
    last_friday_future = None
    for i in range(24, 31):  # Check from 23 to 30 days in the future
        check_date = given_date + timedelta(days=i)
        if check_date.weekday() == 4:  # 4 represents Friday
            last_friday_future = check_date
            break

    # Calculate the first Friday outside 30 days from the given date
    first_friday_outside_30_days = None
    for i in range(31, 38):  # Check from 31 to 37 days in the future
        check_date = given_date + timedelta(days=i)
        if check_date.weekday() == 4:  # 4 represents Friday
            first_friday_outside_30_days = check_date
            break

    # Calculate the differences in days
    days_to_last_friday_future = (last_friday_future - given_date).days
    days_to_first_friday_outside_30 = (first_friday_outside_30_days - given_date).days

    return days_to_last_friday_future, days_to_first_friday_outside_30

def get_option_data(symbol, date):
    near_term_dte, next_term_dte = calculate_friday_differences(date)

    while True:
        try:
            call_data_near_term = getMarketData(
                symbol=symbol,
                tradeDate=date,
                dteFrom=near_term_dte,
                dteTo=near_term_dte,
                deltaFrom=-1000,
                deltaTo=1000,
                cp='C'
            )
            if not call_data_near_term.empty:
                break

        except pd.errors.EmptyDataError:
            near_term_dte -= 1

        except Exception as e:
            print(f'An error occured: {e}')


    put_data_near_term = getMarketData(
        symbol=symbol,
        tradeDate=date,
        dteFrom=near_term_dte,
        dteTo=near_term_dte,
        deltaFrom=-1000,
        deltaTo=1000,
        cp='P'
    )


    while True:
        try:
            call_data_next_term = getMarketData(
                symbol=symbol,
                tradeDate=date,
                dteFrom=next_term_dte,
                dteTo=next_term_dte,
                deltaFrom=-1000,
                deltaTo=1000,
                cp='C'
            )
            if not call_data_next_term.empty:
                break

        except pd.errors.EmptyDataError:
            next_term_dte -= 1

        except Exception as e:
            print(f'An error occured: {e}')
        
    put_data_next_term = getMarketData(
        symbol=symbol,
        tradeDate=date,
        dteFrom=next_term_dte,
        dteTo=next_term_dte,
        deltaFrom=-1000,
        deltaTo=1000,
        cp='P'
    )
    
    return call_data_near_term, put_data_near_term, call_data_next_term, put_data_next_term

def calc_K0(call_data_near_term, put_data_near_term, call_data_next_term, put_data_next_term, r1, r2, T1, T2):
    '''
    call_data_near_term['diff'] = abs(call_data_near_term['price'] - put_data_near_term['price'])
    call_data_next_term['diff'] = abs(call_data_next_term['price'] - put_data_next_term['price'])
    '''
    
    # Filter rows where both prices are not zero
    valid_near_term = (call_data_near_term['price'] != 0) & (put_data_near_term['price'] != 0)
    call_data_near_term['diff'] = float('inf')  # Initialize with a high value to exclude invalid rows
    call_data_near_term.loc[valid_near_term, 'diff'] = abs(call_data_near_term.loc[valid_near_term, 'price'] - put_data_near_term.loc[valid_near_term, 'price'])
    
    # Filter rows where both prices are not zero for next term
    valid_next_term = (call_data_next_term['price'] != 0) & (put_data_next_term['price'] != 0)
    call_data_next_term['diff'] = float('inf')  # Initialize with a high value to exclude invalid rows
    call_data_next_term.loc[valid_next_term, 'diff'] = abs(call_data_next_term.loc[valid_next_term, 'price'] - put_data_next_term.loc[valid_next_term, 'price'])

    strike_near_term = call_data_near_term.loc[call_data_near_term[call_data_near_term['diff'] != 0]['diff'].idxmin(), 'price_strike']
    strike_next_term = call_data_next_term.loc[call_data_next_term[call_data_next_term['diff'] != 0]['diff'].idxmin(), 'price_strike']

    forward_near_term = (strike_near_term + np.exp(r1*T1) * 
                         (call_data_near_term.loc[call_data_near_term['price_strike']==strike_near_term, 'price'].values[0] - 
                                                            put_data_near_term.loc[put_data_near_term['price_strike']==strike_near_term, 'price'].values[0]))
    forward_next_term = (strike_next_term + np.exp(r2*T2) * 
                         (call_data_next_term.loc[call_data_next_term['price_strike']==strike_next_term, 'price'].values[0] - 
                                                            put_data_next_term.loc[put_data_next_term['price_strike']==strike_next_term, 'price'].values[0]))

    K0_near_term = call_data_near_term[call_data_near_term['price_strike'] <= forward_near_term]['price_strike'].max()
    K0_next_term = call_data_next_term[call_data_next_term['price_strike'] <= forward_next_term]['price_strike'].max()
    
    return forward_near_term, forward_next_term, K0_near_term, K0_next_term

def construct_dataframe(call_data_near_term, put_data_near_term, call_data_next_term, put_data_next_term, K0_near_term, K0_next_term):

    filtered_call_near_term = call_data_near_term[call_data_near_term['price_strike']>=K0_near_term].reset_index(drop=True)
    filtered_put_near_term = put_data_near_term[put_data_near_term['price_strike']<=K0_near_term].reset_index(drop=True)

    filtered_call_next_term = call_data_next_term[call_data_next_term['price_strike']>=K0_next_term].reset_index(drop=True)
    filtered_put_next_term = put_data_next_term[put_data_next_term['price_strike']<=K0_next_term].reset_index(drop=True)

    cutoff_put_near_term = len(filtered_put_near_term)
    put_found_near = False
    
    for i in range(len(filtered_put_near_term) - 2, 0, -1):
        if filtered_put_near_term.loc[i, "Bid"] == 0 and filtered_put_near_term.loc[i - 1, "Bid"] == 0:
            cutoff_put_near_term = i + 1
            put_found_near = True
            break
    
    if put_found_near:
        result_put_near = filtered_put_near_term.iloc[cutoff_put_near_term:][['call_put', 'price_strike', 'price']]
    else:
        result_put_near = filtered_put_near_term[['call_put', 'price_strike', 'price']]
    
    cutoff_call_near_term = len(filtered_call_near_term)
    call_found_near = False
    
    for i in range(1, len(filtered_call_near_term) - 1):
        if filtered_call_near_term.loc[i, "Bid"] == 0 and filtered_call_near_term.loc[i + 1, "Bid"] == 0:
            cutoff_call_near_term = i
            call_found_near = True
            break
    
    if call_found_near:
        result_call_near = filtered_call_near_term.iloc[:cutoff_call_near_term][['call_put', 'price_strike', 'price']]
    else:
        result_call_near = filtered_call_near_term[['call_put', 'price_strike', 'price']]
    
    combined_df_near = pd.concat([result_call_near, result_put_near], ignore_index=True)
    
    row_K0_call_near = result_call_near[result_call_near["price_strike"] == K0_near_term]
    row_K0_put_near = result_put_near[result_put_near["price_strike"] == K0_near_term]
    
    average_price_near = (row_K0_call_near["price"].values[0] + row_K0_put_near["price"].values[0]) / 2
    row_K0_combined_near = pd.DataFrame({
        "call_put": ["P/C Average"],
        "price_strike": [K0_near_term],
        "price": [average_price_near]
    })
    
    combined_df_near = combined_df_near[combined_df_near["price_strike"] != K0_near_term]
    combined_df_near = pd.concat([combined_df_near, row_K0_combined_near], ignore_index=True)
    combined_df_near = combined_df_near.sort_values(by="price_strike").reset_index(drop=True)

    cutoff_put_next_term = len(filtered_put_next_term)
    put_found_next = False
    
    for i in range(len(filtered_put_next_term) - 2, 0, -1):
        if filtered_put_next_term.loc[i, "Bid"] == 0 and filtered_put_next_term.loc[i - 1, "Bid"] == 0:
            cutoff_put_next_term = i + 1
            put_found_next = True
            break
    
    if put_found_next:
        result_put_next = filtered_put_next_term.iloc[cutoff_put_next_term:][['call_put', 'price_strike', 'price']]
    else:
        result_put_next = filtered_put_next_term[['call_put', 'price_strike', 'price']]
    
    cutoff_call_next_term = len(filtered_call_next_term)
    call_found_next = False
    
    for i in range(1, len(filtered_call_next_term) - 1):
        if filtered_call_next_term.loc[i, "Bid"] == 0 and filtered_call_next_term.loc[i + 1, "Bid"] == 0:
            cutoff_call_next_term = i
            call_found_next = True
            break
    
    if call_found_next:
        result_call_next = filtered_call_next_term.iloc[:cutoff_call_next_term][['call_put', 'price_strike', 'price']]
    else:
        result_call_next = filtered_call_next_term[['call_put', 'price_strike', 'price']]
    
    combined_df_next = pd.concat([result_call_next, result_put_next], ignore_index=True)
    
    row_K0_call_next = result_call_next[result_call_next["price_strike"] == K0_next_term]
    row_K0_put_next = result_put_next[result_put_next["price_strike"] == K0_next_term]
    
    average_price_next = (row_K0_call_next["price"].values[0] + row_K0_put_next["price"].values[0]) / 2
    row_K0_combined_next = pd.DataFrame({
        "call_put": ["P/C Average"],
        "price_strike": [K0_next_term],
        "price": [average_price_next]
    })
    
    combined_df_next = combined_df_next[combined_df_next["price_strike"] != K0_next_term]
    combined_df_next = pd.concat([combined_df_next, row_K0_combined_next], ignore_index=True)
    combined_df_next = combined_df_next.sort_values(by="price_strike").reset_index(drop=True)

    return combined_df_near, combined_df_next

def calc_contribution(df_near_term, df_next_term, r1, r2, T1, T2):
    price_strike_near = df_near_term['price_strike']
    price_strike_next = df_next_term['price_strike']

    delta_K_near = []
    delta_K_next = []

    for i in range(len(price_strike_near)):
        if i == 0: 
            delta_K_near.append(abs(price_strike_near[i + 1] - price_strike_near[i]))
        elif i == len(price_strike_near) - 1: 
            delta_K_near.append(abs(price_strike_near[i] - price_strike_near[i - 1]))
        else:  
            delta_K_near.append(abs(price_strike_near[i + 1] - price_strike_near[i - 1]) / 2)

    for j in range(len(price_strike_next)):
        if j == 0: 
            delta_K_next.append(abs(price_strike_next[j + 1] - price_strike_next[j]))
        elif j == len(price_strike_next) - 1: 
            delta_K_next.append(abs(price_strike_next[j] - price_strike_next[j - 1]))
        else:  
            delta_K_next.append(abs(price_strike_next[j + 1] - price_strike_next[j - 1]) / 2)

    df_near_term['delta_K'] = delta_K_near
    df_next_term['delta_K'] = delta_K_next

    df_near_term['contribution'] = (df_near_term['delta_K'] / (df_near_term['price_strike'] ** 2) * np.exp(r1 * T1) * df_near_term['price'])
    df_next_term['contribution'] = (df_next_term['delta_K'] / (df_next_term['price_strike'] ** 2) * np.exp(r2 * T2) * df_next_term['price'])

    return df_near_term, df_next_term

def calc_total_contribution(df_near_term, df_next_term, T1, T2):
    
    result_near = 2 / T1 * df_near_term['contribution'].sum()
    result_next = 2 / T2 * df_next_term['contribution'].sum()

    return result_near, result_next

def calc_put_call_contribution(df_near_term, df_next_term, T1, T2):

    call_num_near = df_near_term[df_near_term['call_put'].isin(['C', 'P/C Average'])].shape[0]
    call_num_next = df_next_term[df_next_term['call_put'].isin(['C', 'P/C Average'])].shape[0]
    put_num_near = df_near_term[df_near_term['call_put'].isin(['P', 'P/C Average'])].shape[0]
    put_num_next = df_next_term[df_next_term['call_put'].isin(['P', 'P/C Average'])].shape[0]

    call_ratio_near = call_num_near / df_near_term.shape[0]
    call_ratio_next = call_num_next / df_next_term.shape[0]
    put_ratio_near = put_num_near / df_near_term.shape[0]
    put_ratio_next = put_num_next / df_next_term.shape[0]

    call_contributions_near = df_near_term[df_near_term['call_put'].isin(['C', 'P/C Average'])]['contribution']
    put_contributions_near = df_near_term[df_near_term['call_put'].isin(['P', 'P/C Average'])]['contribution']

    call_contributions_next = df_next_term[df_next_term['call_put'].isin(['C', 'P/C Average'])]['contribution']
    put_contributions_next = df_next_term[df_next_term['call_put'].isin(['P', 'P/C Average'])]['contribution']

    call_near_result = 2 / T1 * call_contributions_near.sum()
    put_near_result = 2 / T1 * put_contributions_near.sum()

    call_next_result = 2 / T2 * call_contributions_next.sum()
    put_next_result = 2 / T2 * put_contributions_next.sum()
    
    return call_near_result, put_near_result, call_next_result, put_next_result, call_ratio_near, call_ratio_next, put_ratio_near, put_ratio_next

def calc_total_sigma(result_near, result_next, forward_near_term, forward_next_term, K0_near_term, K0_next_term, T1, T2):
    
    sigma1 = result_near - (forward_near_term/K0_near_term - 1)**2 / T1
    sigma2 = result_next - (forward_next_term/K0_next_term - 1)**2 / T2

    return sigma1, sigma2

def calc_call_put_sigma(call_near_result, put_near_result, call_next_result, put_next_result, call_ratio_near, call_ratio_next, put_ratio_near, put_ratio_next, forward_near_term, forward_next_term, K0_near_term, K0_next_term, T1, T2):

    sigma1_call = call_near_result - call_ratio_near * (forward_near_term/K0_near_term - 1)**2 / T1
    sigma2_call = call_next_result - call_ratio_next * (forward_next_term/K0_next_term - 1)**2 / T2

    sigma1_put = put_near_result - put_ratio_near * (forward_near_term/K0_near_term - 1)**2 / T1
    sigma2_put = put_next_result - put_ratio_next * (forward_next_term/K0_next_term - 1)**2 / T2

    return sigma1_call, sigma2_call, sigma1_put, sigma2_put

def calc_vix(sigma1, sigma2, T1, T2, M_T1, M_T2, M_CM):
    vix = 100 * np.sqrt((T1 * sigma1 * (M_T2-M_CM)/(M_T2-M_T1) + T2 * sigma2 * (M_CM-M_T1)/(M_T2-M_T1)) * 365/M_CM)

    return vix

def calc_call_put_vix(sigma1_call, sigma2_call, sigma1_put, sigma2_put, T1, T2, M_T1, M_T2, M_CM):
    vix_call = 100 * np.sqrt((T1 * sigma1_call * (M_T2-M_CM)/(M_T2-M_T1) + T2 * sigma2_call * (M_CM-M_T1)/(M_T2-M_T1)) * 365/M_CM)
    vix_put = 100 * np.sqrt((T1 * sigma1_put * (M_T2-M_CM)/(M_T2-M_T1) + T2 * sigma2_put * (M_CM-M_T1)/(M_T2-M_T1)) * 365/M_CM)

    return vix_call, vix_put

def calc_VIX(symbol, date, r1, r2, M_CM):

    M_T1, M_T2 = calculate_friday_differences(date)
    T1 = M_T1 / 365
    T2 = M_T2 / 365
    
    call_data_near_term, put_data_near_term, call_data_next_term, put_data_next_term = get_option_data(symbol, date)

    year, month, _ = date.split("-")
    year_folder = os.path.join("SPX_Option_Data", year)
    month_folder = os.path.join(year_folder, month)
    os.makedirs(month_folder, exist_ok=True)
    
    # Save dataframes into the month folder with descriptive filenames
    call_data_near_term.to_excel(os.path.join(month_folder, f"{date}_call_data_near_term.xlsx"), index=False)
    put_data_near_term.to_excel(os.path.join(month_folder, f"{date}_put_data_near_term.xlsx"), index=False)
    call_data_next_term.to_excel(os.path.join(month_folder, f"{date}_call_data_next_term.xlsx"), index=False)
    put_data_next_term.to_excel(os.path.join(month_folder, f"{date}_put_data_next_term.xlsx"), index=False)
    

    forward_near_term, forward_next_term, K0_near_term, K0_next_term = calc_K0(call_data_near_term, put_data_near_term, call_data_next_term, put_data_next_term, r1, r2, T1, T2)

    combined_df_near, combined_df_next = construct_dataframe(call_data_near_term, put_data_near_term, call_data_next_term, put_data_next_term, K0_near_term, K0_next_term)

    df_near_term, df_next_term = calc_contribution(combined_df_near, combined_df_next, r1, r2, T1, T2)
    

    result_near, result_next = calc_total_contribution(df_near_term, df_next_term, T1, T2)

    sigma1, sigma2 = calc_total_sigma(result_near, result_next, forward_near_term, forward_next_term, K0_near_term, K0_next_term, T1, T2)

    vix = calc_vix(sigma1, sigma2, T1, T2, M_T1, M_T2, M_CM)


    call_near_result, put_near_result, call_next_result, put_next_result, call_ratio_near, call_ratio_next, put_ratio_near, put_ratio_next = calc_put_call_contribution(df_near_term, df_next_term, T1, T2)

    sigma1_call, sigma2_call, sigma1_put, sigma2_put = calc_call_put_sigma(call_near_result, put_near_result, call_next_result, put_next_result, call_ratio_near, call_ratio_next, put_ratio_near, put_ratio_next, forward_near_term, forward_next_term, K0_near_term, K0_next_term, T1, T2)

    vix_call, vix_put = calc_call_put_vix(sigma1_call, sigma2_call, sigma1_put, sigma2_put, T1, T2, M_T1, M_T2, M_CM)

    return date, vix, vix_call, vix_put

        

In [10]:
def get_all_trade_dates(start_date="2009-01-01", end_date="2010-01-01"):
    """
    Retrieve all trading dates of SPX within a given date range.
    
    Parameters:
    - start_date (str): Start date in the format "YYYY-MM-DD". Default is "2014-08-30".
    - end_date (str): End date in the format "YYYY-MM-DD". Default is "2024-08-31".
    
    Returns:
    - set: A set of trading dates in the format "YYYY-MM-DD".
    """
    ticker = yf.Ticker("^GSPC")  # Use the correct ticker for S&P 500 (SPX)
    history = ticker.history(start=start_date, end=end_date, interval="1d")
    trade_dates = set(history.index.strftime("%Y-%m-%d"))
    return trade_dates

# Define the set of trading dates within the specified range
TRADE_DATES = get_all_trade_dates()

In [11]:
sorted_trade_dates = sorted(TRADE_DATES) if 'TRADE_DATES' in globals() else []

In [12]:
sorted_trade_dates

['2009-01-02',
 '2009-01-05',
 '2009-01-06',
 '2009-01-07',
 '2009-01-08',
 '2009-01-09',
 '2009-01-12',
 '2009-01-13',
 '2009-01-14',
 '2009-01-15',
 '2009-01-16',
 '2009-01-20',
 '2009-01-21',
 '2009-01-22',
 '2009-01-23',
 '2009-01-26',
 '2009-01-27',
 '2009-01-28',
 '2009-01-29',
 '2009-01-30',
 '2009-02-02',
 '2009-02-03',
 '2009-02-04',
 '2009-02-05',
 '2009-02-06',
 '2009-02-09',
 '2009-02-10',
 '2009-02-11',
 '2009-02-12',
 '2009-02-13',
 '2009-02-17',
 '2009-02-18',
 '2009-02-19',
 '2009-02-20',
 '2009-02-23',
 '2009-02-24',
 '2009-02-25',
 '2009-02-26',
 '2009-02-27',
 '2009-03-02',
 '2009-03-03',
 '2009-03-04',
 '2009-03-05',
 '2009-03-06',
 '2009-03-09',
 '2009-03-10',
 '2009-03-11',
 '2009-03-12',
 '2009-03-13',
 '2009-03-16',
 '2009-03-17',
 '2009-03-18',
 '2009-03-19',
 '2009-03-20',
 '2009-03-23',
 '2009-03-24',
 '2009-03-25',
 '2009-03-26',
 '2009-03-27',
 '2009-03-30',
 '2009-03-31',
 '2009-04-01',
 '2009-04-02',
 '2009-04-03',
 '2009-04-06',
 '2009-04-07',
 '2009-04-

In [ ]:
# Apply `calc_VIX` to all dates in TRADE_DATES
# Timer setup
start_time = time.time()
results = []

for idx, date in enumerate(sorted_trade_dates):
    loop_start = time.time()
    print(f"Processing date {idx + 1}/{len(sorted_trade_dates)}: {date}")
    
    result = calc_VIX('SPX', date, 0.03, 0.035, 30)
    results.append(result)
    
    # Calculate and display elapsed and estimated remaining time
    loop_end = time.time()
    elapsed = loop_end - start_time
    per_iteration = loop_end - loop_start
    remaining = per_iteration * (len(sorted_trade_dates) - idx - 1)
    print(f"Time for this iteration: {per_iteration:.2f}s | Elapsed: {elapsed:.2f}s | Estimated remaining: {remaining:.2f}s")

# End timer and print total time
end_time = time.time()
total_time = end_time - start_time
print(f"Total time: {total_time:.2f}s")

Processing date 1/252: 2009-01-02
Time for this iteration: 56.08s | Elapsed: 56.08s | Estimated remaining: 14076.60s
Processing date 2/252: 2009-01-05
Time for this iteration: 97.33s | Elapsed: 153.41s | Estimated remaining: 24332.53s
Processing date 3/252: 2009-01-06
Time for this iteration: 48.09s | Elapsed: 201.50s | Estimated remaining: 11975.03s
Processing date 4/252: 2009-01-07
Time for this iteration: 62.23s | Elapsed: 263.73s | Estimated remaining: 15432.75s
Processing date 5/252: 2009-01-08
Time for this iteration: 75.56s | Elapsed: 339.30s | Estimated remaining: 18663.69s
Processing date 6/252: 2009-01-09
Time for this iteration: 108.88s | Elapsed: 448.17s | Estimated remaining: 26783.38s
Processing date 7/252: 2009-01-12
Time for this iteration: 94.67s | Elapsed: 542.84s | Estimated remaining: 23193.91s
Processing date 8/252: 2009-01-13
Time for this iteration: 78.31s | Elapsed: 621.15s | Estimated remaining: 19108.36s
Processing date 9/252: 2009-01-14
Time for this iteratio

### Key Event on 2020-02-27 (Missing Data)
### Key Event on 2015-08-28 (Outlier)
### Key Event on 2017-07-13 (Outlier)

In [5]:
base_folder="SPX_Option_Data"
os.makedirs(base_folder, exist_ok=True)

In [99]:
results

[('2024-04-19', 18.723295044747584, 10.52132959830138, 15.632537042231293),
 ('2024-04-22', 16.926153109150953, 9.859652651388066, 13.906401129355903),
 ('2024-04-23', 15.681355542886713, 9.183322293979085, 12.857314976776127),
 ('2024-04-24', 15.953780343648466, 9.160013029048656, 13.20772555450523),
 ('2024-04-25', 15.353379081711044, 8.695793255988589, 12.79631464371311),
 ('2024-04-26', 15.01176950536054, 8.61350560243615, 12.43755142383371),
 ('2024-04-29', 14.658353291249487, 8.362771414664696, 12.180425517495296),
 ('2024-04-30', 15.632841862763325, 9.026593317436056, 12.910408229704053),
 ('2024-05-01', 15.373297248806239, 9.078948745407697, 12.557175345124783),
 ('2024-05-02', 14.638971388558058, 8.795407389074755, 11.854907219847144),
 ('2024-05-03', 13.47132101375169, 8.096505440885384, 10.91536709180577),
 ('2024-05-06', 13.471977694752585, 8.055024707486327, 10.945075594511366),
 ('2024-05-07', 13.213390753825339, 7.833723821049878, 10.784244470280372),
 ('2024-05-08', 12.

In [30]:
df_100104_100129 = results.copy()

In [100]:
df = pd.read_excel("Calculated_VIX.xlsx")

In [101]:
df

,Date,VIX,Call_contribution,Put_contribution
0,2010-01-04,11.744340,7.403955,9.755545
1,2010-01-05,10.426754,6.558768,8.740827
2,2010-01-06,9.813317,6.436661,8.065107
3,2010-01-07,9.430135,6.199992,7.769425
4,2010-01-08,7.851250,5.176900,6.552607
...,...,...,...,...
3592,2024-04-12,17.289118,9.355988,14.673893
3593,2024-04-15,19.205120,10.479543,16.233317
3594,2024-04-16,18.396439,10.365330,15.342557
3595,2024-04-17,18.220601,10.099978,15.308264


In [102]:
list_of_rows = df.values.tolist()

In [103]:
data_tuples = [tuple(row) for row in list_of_rows]

In [104]:
data_tuples

[('2010-01-04', 11.74433986537582, 7.403955083547789, 9.755544915359451),
 ('2010-01-05', 10.42675436213889, 6.558767802723006, 8.74082724169154),
 ('2010-01-06', 9.813317175235984, 6.436660767487036, 8.065106538158236),
 ('2010-01-07', 9.430135212356658, 6.19999155515911, 7.769425291606217),
 ('2010-01-08', 7.851250014929841, 5.176899719962393, 6.552607331893593),
 ('2010-01-11', 6.663754609108734, 4.591185819754572, 5.498646917111365),
 ('2010-01-12', 6.169683033118416, 4.325652237933729, 5.077377286937013),
 ('2010-01-13', 5.173370100160469, 3.390237083279894, 4.48412287408819),
 ('2010-01-14', 3.554347550453667, 2.990701817456394, 2.809780557562752),
 ('2010-01-15', 7.963270559781954, 7.235922763063725, 7.40876011228652),
 ('2010-01-19', 4.294506892589178, 5.423538398872727, 3.402466450646211),
 ('2010-01-20', 4.748613270131584, 4.050079318803798, 4.524926823752204),
 ('2010-01-21', 9.284582368521379, 6.834419741218997, 8.51775712604707),
 ('2010-01-22', 18.92853292866505, 13.40716

In [105]:
v = data_tuples + results

In [106]:
vix_df = pd.DataFrame(v, columns=['Date', 'VIX', 'Call_contribution', 'Put_contribution'])

In [107]:
vix_df.to_excel("Calculated_VIX.xlsx", index=False)

In [120]:
calc_VIX('SPX', '2005-01-03', 0.03, 0.035, 30)

('2005-01-03', 10.985159982921033, 7.829135702059295, 8.405279533488033)

In [7]:
calc_VIX('SPX', '2017-07-13', 0.03, 0.035, 30)

('2017-07-13', 9.952752109961416, 4.914003786388064, 8.894932051340827)

In [8]:
calc_VIX('SPX', '2024-08-05', 0.03, 0.035, 30)

('2024-08-05', 38.49563634824241, 16.180327101373827, 35.03800001643828)

In [104]:
M_T1, M_T2 = calculate_friday_differences('2016-07-15')
T1 = M_T1 / 365
T2 = M_T2 / 365
r1 = 0.03
r2 = 0.035

In [105]:
call_data_near_term, put_data_near_term, call_data_next_term, put_data_next_term = get_option_data('SPX', '2016-07-15')

In [106]:
forward_near_term, forward_next_term, K0_near_term, K0_next_term = calc_K0(call_data_near_term, put_data_near_term, call_data_next_term, put_data_next_term, r1, r2, T1, T2)

In [107]:
df_near_term, df_next_term = construct_dataframe(call_data_near_term, put_data_near_term, call_data_next_term, put_data_next_term, K0_near_term, K0_next_term)

In [108]:
df_near_term

,call_put,price_strike,price
0,P,1275.0,0.075000
1,P,1300.0,0.075000
2,P,1325.0,0.075000
3,P,1350.0,0.075000
4,P,1375.0,0.000000
5,P,1400.0,0.100000
6,P,1425.0,0.100000
7,P,1450.0,0.150000
8,P,1475.0,0.175000
9,P,1500.0,0.125000


In [109]:
df_next_term

,call_put,price_strike,price
0,P,1430.0,0.150000
1,P,1435.0,0.000000
2,P,1435.0,0.150000
3,P,1440.0,0.150000
4,P,1440.0,0.000000
5,P,1445.0,0.150000
6,P,1445.0,0.000000
7,P,1450.0,0.100000
8,P,1450.0,0.150000
9,P,1455.0,0.175000


In [90]:
a,b = calc_contribution(df_near_term, df_next_term, r1, r2, T1, T2)

In [91]:
a

,call_put,price_strike,price,delta_K,contribution
0,P,900.0,0.150000,50.0,9.281356e-06
1,P/C Average,950.0,519.300000,50.0,2.883874e-02
2,C,1000.0,0.000000,50.0,0.000000e+00
3,C,1050.0,938.500000,50.0,4.266393e-02
4,C,1100.0,888.500000,50.0,3.680249e-02
5,C,1150.0,838.500000,37.5,2.383273e-02
6,C,1175.0,813.500000,25.0,1.476580e-02
7,C,1200.0,788.500000,25.0,1.372190e-02
8,C,1225.0,763.600000,25.0,1.275172e-02
9,C,1250.0,738.600000,25.0,1.184580e-02


In [92]:
b

,call_put,price_strike,price,delta_K,contribution
0,P,800.0,0.225000,50.0,1.763891e-05
1,P,850.0,0.225000,50.0,1.562478e-05
2,P,900.0,0.200000,50.0,1.238837e-05
3,P,950.0,0.250000,50.0,1.389831e-05
4,P,1000.0,0.275000,50.0,1.379755e-05
5,P,1050.0,0.325000,50.0,1.479020e-05
6,P,1100.0,0.325000,50.0,1.347619e-05
7,P,1150.0,0.425000,37.5,1.209271e-05
8,P,1175.0,0.500000,25.0,9.085179e-06
9,P,1200.0,0.625000,25.0,1.088822e-05


In [95]:
forward_near_term

1988.5465427205106

In [96]:
forward_next_term

1987.742216444631

In [97]:
K0_near_term

1985.0

In [98]:
K0_next_term

1985.0

In [86]:
call_data_near_term

,c_date,option_symbol,dte,stocks_id,expiration_date,call_put,price_strike,price_open,price_high,price_low,price,volume,openinterest,iv,delta,preiv,gamma,theta,vega,rho,Ask,Bid,underlying_price,calc_OTM,option_id,diff
0,2015-08-28 00:00:00.0,SPXW 150925C00600000,28,9327,2015-09-25 00:00:00.0,C,600.0,NaN,NaN,NaN,1388.300000,0,0,0.441026,0.998433,-1.000000,0.000000,0.000000,0.000000,0.460203,1397.90,1378.70,1991.686,-69.87,NaN,1388.300000
1,2015-08-28 00:00:00.0,SPXW 150925C00650000,28,9327,2015-09-25 00:00:00.0,C,650.0,NaN,NaN,NaN,1338.300000,0,0,0.441026,0.998433,-1.000000,0.000000,0.000000,0.000000,0.498554,1347.90,1328.70,1991.686,-67.36,NaN,1338.300000
2,2015-08-28 00:00:00.0,SPXW 150925C00700000,28,9327,2015-09-25 00:00:00.0,C,700.0,NaN,NaN,NaN,1288.650000,0,0,0.441026,0.998433,-1.000000,0.000000,0.000000,0.000000,0.536904,1300.50,1276.80,1991.686,-64.85,NaN,1288.650000
3,2015-08-28 00:00:00.0,SPXW 150925C00750000,28,9327,2015-09-25 00:00:00.0,C,750.0,NaN,NaN,NaN,1238.300000,0,10,0.441026,0.998433,-1.000000,0.000000,0.000000,0.000000,0.575254,1247.90,1228.70,1991.686,-62.34,NaN,1238.300000
4,2015-08-28 00:00:00.0,SPXW 150925C00800000,28,9327,2015-09-25 00:00:00.0,C,800.0,NaN,NaN,NaN,1188.300000,0,10,0.441026,0.998433,-1.000000,0.000000,0.000000,0.000000,0.613605,1197.90,1178.70,1991.686,-59.83,NaN,1188.300000
5,2015-08-28 00:00:00.0,SPXW 150925C00850000,28,9327,2015-09-25 00:00:00.0,C,850.0,NaN,NaN,NaN,1138.400000,0,1,0.441026,0.998433,-1.000000,0.000000,0.000000,0.000000,0.651955,1148.00,1128.80,1991.686,-57.32,NaN,1138.400000
6,2015-08-28 00:00:00.0,SPXW 150925C00900000,28,9327,2015-09-25 00:00:00.0,C,900.0,NaN,NaN,NaN,1088.400000,0,0,0.441026,0.998433,-1.000000,0.000000,0.000000,0.000000,0.690305,1098.00,1078.80,1991.686,-54.81,NaN,1088.250000
7,2015-08-28 00:00:00.0,SPXW 150925C00950000,28,9327,2015-09-25 00:00:00.0,C,950.0,NaN,NaN,NaN,1038.400000,0,0,0.441026,0.998433,-1.000000,0.000000,0.000000,0.000000,0.728655,1048.00,1028.80,1991.686,-52.30,NaN,1038.200000
8,2015-08-28 00:00:00.0,SPXW 150925C01000000,28,9327,2015-09-25 00:00:00.0,C,1000.0,NaN,NaN,NaN,0.000000,0,0,0.441026,0.998433,-1.000000,0.000000,0.000000,0.000000,0.767006,998.00,978.80,1991.686,-49.79,NaN,0.150000
9,2015-08-28 00:00:00.0,SPXW 150925C01050000,28,9327,2015-09-25 00:00:00.0,C,1050.0,NaN,NaN,NaN,938.500000,0,0,0.441026,0.998433,-1.000000,0.000000,0.000000,0.000002,0.805356,948.10,928.90,1991.686,-47.28,NaN,938.500000


In [87]:
put_data_near_term

,c_date,option_symbol,dte,stocks_id,expiration_date,call_put,price_strike,price_open,price_high,price_low,price,volume,openinterest,iv,delta,preiv,gamma,theta,vega,rho,Ask,Bid,underlying_price,calc_OTM,option_id
0,2015-08-28 00:00:00.0,SPXW 150925P00600000,28,9327,2015-09-25 00:00:00.0,P,600.0,NaN,NaN,NaN,0.000000,0,0,0.556349,-0.000000,-1.000000,0.000000,-0.000000,0.000000,-0.000000,0.25,0.00,1991.686,69.87,NaN
1,2015-08-28 00:00:00.0,SPXW 150925P00650000,28,9327,2015-09-25 00:00:00.0,P,650.0,NaN,NaN,NaN,0.000000,0,0,0.556349,-0.000000,-1.000000,0.000000,-0.000000,0.000000,-0.000000,0.25,0.00,1991.686,67.36,NaN
2,2015-08-28 00:00:00.0,SPXW 150925P00700000,28,9327,2015-09-25 00:00:00.0,P,700.0,NaN,NaN,NaN,0.000000,0,0,0.556349,-0.000000,-1.000000,0.000000,-0.000000,0.000000,-0.000000,0.30,0.00,1991.686,64.85,NaN
3,2015-08-28 00:00:00.0,SPXW 150925P00750000,28,9327,2015-09-25 00:00:00.0,P,750.0,NaN,NaN,NaN,0.000000,0,1433,0.556349,-0.000000,-1.000000,0.000000,-0.000000,0.000000,-0.000000,0.30,0.00,1991.686,62.34,NaN
4,2015-08-28 00:00:00.0,SPXW 150925P00800000,28,9327,2015-09-25 00:00:00.0,P,800.0,NaN,NaN,NaN,0.000000,0,271,0.556349,-0.000000,-1.000000,0.000000,-0.000000,0.000000,-0.000000,0.25,0.00,1991.686,59.83,NaN
5,2015-08-28 00:00:00.0,SPXW 150925P00850000,28,9327,2015-09-25 00:00:00.0,P,850.0,NaN,NaN,NaN,0.000000,0,1155,0.556349,-0.000000,-1.000000,0.000000,-0.000000,0.000000,-0.000000,0.25,0.00,1991.686,57.32,NaN
6,2015-08-28 00:00:00.0,SPXW 150925P00900000,28,9327,2015-09-25 00:00:00.0,P,900.0,0.15,0.15,0.10,0.150000,305,1961,0.556349,-0.000000,-1.000000,0.000000,-0.000003,0.000003,-0.000000,0.20,0.10,1991.686,54.81,NaN
7,2015-08-28 00:00:00.0,SPXW 150925P00950000,28,9327,2015-09-25 00:00:00.0,P,950.0,NaN,NaN,NaN,0.200000,0,2214,0.556349,-0.000001,-1.000000,0.000000,-0.000015,0.000015,-0.000001,0.30,0.10,1991.686,52.30,NaN
8,2015-08-28 00:00:00.0,SPXW 150925P01000000,28,9327,2015-09-25 00:00:00.0,P,1000.0,0.20,0.20,0.20,0.150000,500,6518,0.556349,-0.000003,-1.000000,0.000000,-0.000074,0.000074,-0.000004,0.25,0.05,1991.686,49.79,NaN
9,2015-08-28 00:00:00.0,SPXW 150925P01050000,28,9327,2015-09-25 00:00:00.0,P,1050.0,0.20,0.20,0.20,0.000000,1,4056,0.556349,-0.000012,-1.000000,0.000000,-0.000295,0.000295,-0.000019,0.30,0.00,1991.686,47.28,NaN


In [88]:
call_data_next_term

,c_date,option_symbol,dte,stocks_id,expiration_date,call_put,price_strike,price_open,price_high,price_low,price,volume,openinterest,iv,delta,preiv,gamma,theta,vega,rho,Ask,Bid,underlying_price,calc_OTM,option_id,diff
0,2015-08-28 00:00:00.0,SPXW 151002C00600000,35,9327,2015-10-02 00:00:00.0,C,600.0,NaN,NaN,NaN,1387.600000,0,0,0.551939,0.998042,-1.000000,0.000000,0.000000,0.000000,0.575226,1397.20,1378.00,1991.686,-69.87,NaN,1387.600000
1,2015-08-28 00:00:00.0,SPXW 151002C00650000,35,9327,2015-10-02 00:00:00.0,C,650.0,NaN,NaN,NaN,1337.600000,0,0,0.551939,0.998042,-1.000000,0.000000,0.000000,0.000000,0.623161,1347.20,1328.00,1991.686,-67.36,NaN,1337.600000
2,2015-08-28 00:00:00.0,SPXW 151002C00700000,35,9327,2015-10-02 00:00:00.0,C,700.0,NaN,NaN,NaN,1287.600000,0,0,0.551939,0.998042,-1.000000,0.000000,0.000000,0.000000,0.671097,1297.20,1278.00,1991.686,-64.85,NaN,1287.600000
3,2015-08-28 00:00:00.0,SPXW 151002C00750000,35,9327,2015-10-02 00:00:00.0,C,750.0,NaN,NaN,NaN,1237.700000,0,10,0.551939,0.998042,-1.000000,0.000000,0.000000,0.000000,0.719032,1247.30,1228.10,1991.686,-62.34,NaN,1237.700000
4,2015-08-28 00:00:00.0,SPXW 151002C00800000,35,9327,2015-10-02 00:00:00.0,C,800.0,NaN,NaN,NaN,1187.700000,0,10,0.551939,0.998042,-1.000000,0.000000,0.000000,0.000001,0.766968,1197.30,1178.10,1991.686,-59.83,NaN,1187.475000
5,2015-08-28 00:00:00.0,SPXW 151002C00850000,35,9327,2015-10-02 00:00:00.0,C,850.0,NaN,NaN,NaN,1137.700000,0,10,0.551939,0.998042,-1.000000,0.000000,0.000000,0.000007,0.814903,1149.50,1125.90,1991.686,-57.32,NaN,1137.475000
6,2015-08-28 00:00:00.0,SPXW 151002C00900000,35,9327,2015-10-02 00:00:00.0,C,900.0,NaN,NaN,NaN,1087.800000,0,0,0.551939,0.998041,-1.000000,0.000000,0.000000,0.000035,0.862837,1097.40,1078.20,1991.686,-54.81,NaN,1087.600000
7,2015-08-28 00:00:00.0,SPXW 151002C00950000,35,9327,2015-10-02 00:00:00.0,C,950.0,NaN,NaN,NaN,1037.800000,0,0,0.551939,0.998037,-1.000000,0.000000,0.000000,0.000149,0.910764,1047.40,1028.20,1991.686,-52.30,NaN,1037.550000
8,2015-08-28 00:00:00.0,SPXW 151002C01000000,35,9327,2015-10-02 00:00:00.0,C,1000.0,NaN,NaN,NaN,987.900000,0,10,0.551939,0.998022,-1.000000,0.000000,0.000000,0.000535,0.958670,997.50,978.30,1991.686,-49.79,NaN,987.625000
9,2015-08-28 00:00:00.0,SPXW 151002C01050000,35,9327,2015-10-02 00:00:00.0,C,1050.0,NaN,NaN,NaN,937.900000,0,0,0.551939,0.997976,-1.000000,0.000001,0.000000,0.001660,1.006513,947.50,928.30,1991.686,-47.28,NaN,937.575000


In [89]:
put_data_next_term

,c_date,option_symbol,dte,stocks_id,expiration_date,call_put,price_strike,price_open,price_high,price_low,price,volume,openinterest,iv,delta,preiv,gamma,theta,vega,rho,Ask,Bid,underlying_price,calc_OTM,option_id
0,2015-08-28 00:00:00.0,SPXW 151002P00600000,35,9327,2015-10-02 00:00:00.0,P,600.0,NaN,NaN,NaN,0.000000,0,10,0.660701,-0.000000,-1.000000,0.000000,-0.000000,0.000000,-0.000000,0.25,0.00,1991.686,69.87,NaN
1,2015-08-28 00:00:00.0,SPXW 151002P00650000,35,9327,2015-10-02 00:00:00.0,P,650.0,NaN,NaN,NaN,0.000000,0,0,0.660701,-0.000000,-1.000000,0.000000,-0.000000,0.000000,-0.000000,0.30,0.00,1991.686,67.36,NaN
2,2015-08-28 00:00:00.0,SPXW 151002P00700000,35,9327,2015-10-02 00:00:00.0,P,700.0,NaN,NaN,NaN,0.000000,0,0,0.660701,-0.000000,-1.000000,0.000000,-0.000003,0.000003,-0.000000,0.30,0.00,1991.686,64.85,NaN
3,2015-08-28 00:00:00.0,SPXW 151002P00750000,35,9327,2015-10-02 00:00:00.0,P,750.0,0.15,0.15,0.15,0.000000,59,1633,0.660701,-0.000001,-1.000000,0.000000,-0.000017,0.000018,-0.000001,0.35,0.00,1991.686,62.34,NaN
4,2015-08-28 00:00:00.0,SPXW 151002P00800000,35,9327,2015-10-02 00:00:00.0,P,800.0,0.15,0.15,0.15,0.225000,25,1316,0.660701,-0.000003,-1.000000,0.000000,-0.000074,0.000078,-0.000005,0.40,0.05,1991.686,59.83,NaN
5,2015-08-28 00:00:00.0,SPXW 151002P00850000,35,9327,2015-10-02 00:00:00.0,P,850.0,NaN,NaN,NaN,0.225000,0,547,0.660701,-0.000010,-1.000000,0.000000,-0.000272,0.000287,-0.000021,0.40,0.05,1991.686,57.32,NaN
6,2015-08-28 00:00:00.0,SPXW 151002P00900000,35,9327,2015-10-02 00:00:00.0,P,900.0,NaN,NaN,NaN,0.200000,0,350,0.660701,-0.000035,-1.000000,0.000000,-0.000858,0.000906,-0.000070,0.35,0.05,1991.686,54.81,NaN
7,2015-08-28 00:00:00.0,SPXW 151002P00950000,35,9327,2015-10-02 00:00:00.0,P,950.0,NaN,NaN,NaN,0.250000,0,151,0.660701,-0.000103,-1.000000,0.000001,-0.002371,0.002501,-0.000206,0.45,0.05,1991.686,52.30,NaN
8,2015-08-28 00:00:00.0,SPXW 151002P01000000,35,9327,2015-10-02 00:00:00.0,P,1000.0,NaN,NaN,NaN,0.275000,0,202,0.660701,-0.000268,-1.000000,0.000002,-0.005829,0.006147,-0.000540,0.50,0.05,1991.686,49.79,NaN
9,2015-08-28 00:00:00.0,SPXW 151002P01050000,35,9327,2015-10-02 00:00:00.0,P,1050.0,NaN,NaN,NaN,0.325000,0,325,0.660701,-0.000633,-1.000000,0.000005,-0.012937,0.013639,-0.001280,0.55,0.10,1991.686,47.28,NaN


In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)